In [ ]:
import pandas as pd
import numpy as np
import os
import json, requests
import time

In [ ]:
city_country = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/city_country_separate.csv')

In [ ]:
key = os.getenv('G_API_KEY')
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}'
maxprice = 'https://maps.googleapis.com/maps/api/place/textsearch/json?language=en&key={}&query={}&minprice={}&maxprice={}'

In [ ]:
urll = maxprice.format(key, query.format('portland'), 2, 2)

In [ ]:
urll

In [ ]:
query_dict = {'attractions': "{}+points+of+interest",
              'restaurants': "best+restaurants+in+{}",
             'restaurants_price': 'best+restaurants+in+{}&minprice={}&maxprice={}'} #build seperate query for min/max price 
city_list = city_country.iloc[0:2, :].copy()


In [ ]:
url_price = base_url.format(key) + '&query={}'.format(query3.format('portland', 2, 2))

In [ ]:
url_price

In [ ]:
city_list.replace(' ', '+', regex=True, inplace=True)

In [ ]:
city_list

In [ ]:
q = build_query('Portland', query_dict['restaurants_price'])

In [ ]:
def city_query_selection(city_list, query_dict):
    for city in city_list.city:
        for query in query_dict:
            pipeline(city, query_dict[query])
            
    return

def pipeline(city, query):
    url = build_query(city, query)
    data = find_places_api(url)
    df = create_df(data)
    
    return df


def build_query(city, query):
    if 'maxprice' in query:
        k = 1
        while k <= 4:
            url = base_url.format(key) + '&query={}'.format(query.format(city, k, k))
            k += 1 
            # DO I need a return for each k? 
    else:
        url = base_url.format(key) + '&query={}'.format(query.format(city))
    return query


def make_next_url(results, base_url):
  return base_url.format(key) + '&pagetoken={}'.format(results['next_page_token'])

def find_places_api(url, data=[]):
  print(url)
  results = requests.get(url).json()
  time.sleep(3)
  data = data + results['results']
  print(results.keys())
  print(results['status'])
  if 'next_page_token' not in results.keys() and results['status']=='OK':
    print('no next')
    return data
  elif results['status']=='OK':
    print('next')
    new_url = make_next_url(results, base_url=base_url)
    return find_places_api(new_url, data)
  else:
    return data 

def create_df(json_data, city):
    #find_places_api()
    df = pd.json_normalize(json_data)
    print('Inside create df', city)
    city.replace('+', ' ')
    df['city'] = city
    #df = column_selection(df)
    # SEND TO COLUMN SELECTION raw_to_df.py
    return df

def column_selection(df):
    cols_to_keep = ['country', 'city', 'name', 'formatted_address',
    'price_level', 'rating', 'user_ratings_total', 'types',
    'geometry.location.lat', 'geometry.location.lng', 'place_id']
    
    df = df[cols_to_keep]
    df.rename(columns={'formatted_address': 'address',
    'geometry.location.lat': 'latitude', 'geometry.location.lng': 'longitude'}, inplace=True)
    return df
    

    
def table_name():
    
    return

def write_db(table_name, df):
    df.to_sql(table_name, conn, if_exists="append")
    return 
# append data or delete and recreate when rerunning